## Advent of Code - Day 23

### Star 1 & 2

In [1]:
import numpy as np
import heapq
from copy import deepcopy

In [2]:
class Caves:    
    def __init__(self, caves_list, cave_depth):
        self.caves_list = caves_list
        self.caves_pos = [2, 4, 6, 8]
        self.ready = self._ready_caves()
        self.cave_depth = cave_depth
        
    def _ready_caves(self):
        ready = []
        for i, cave in enumerate(self.caves_list):
            ready.append(cave.count(i) == len(cave))
        return ready 

In [3]:
class Burrow:
    def __init__(self, burrow_state, cave_depth):
        self.hall, self._caves_list = burrow_state
        self.legal_hall_pos = np.full(11, True)
        self.legal_hall_pos[np.array([2, 4, 6, 8])] = False
        self.caves = Caves(self._caves_list, cave_depth)
        
    def go_home_moves(self):
        moves = []
        for hall_pos in np.where(self.hall != 0)[0]:
            amphipod_num = int(self.hall[hall_pos]) - 1
            if self.caves.ready[amphipod_num]:
                cave_pos = self.caves.caves_pos[amphipod_num]
                if hall_pos < cave_pos:
                    left = hall_pos + 1
                    right = cave_pos
                else:
                    left = cave_pos
                    right = hall_pos - 1
                if self.hall[left:right+1].sum() == 0:
                    moves.append((f'h.{hall_pos}', f'c.{amphipod_num}'))
        return moves
    
    def go_hall_moves(self):
        moves = []
        for cave in range(4):
            if not self.caves.ready[cave]:
                cave_pos = self.caves.caves_pos[cave]
                free_to_right = np.cumsum(self.hall[cave_pos:], dtype=np.int8) == 0
                free_to_left = np.cumsum(self.hall[:cave_pos][::-1], dtype=np.int8)[::-1] == 0
                free = np.hstack((free_to_left, free_to_right)) & self.legal_hall_pos
                for hall_pos in np.where(free)[0]:
                    moves.append((f'c.{cave}', f'h.{hall_pos}'))

        return moves
    
    def move_cost(self, move):
        start = move[0].split('.')
        end = move[1].split('.')
        if start[0] == 'h':
            x_start = int(start[1])
            cave_num = int(end[1])
            x_end = self.caves.caves_pos[cave_num]
            delta_x = abs(x_start - x_end)
            delta_y = self.caves.cave_depth - len(self.caves.caves_list[cave_num])
            amphipod_num = cave_num
        if start[0] == 'c':
            cave_num = int(start[1])
            x_start = self.caves.caves_pos[cave_num]
            x_end = int(end[1])
            delta_x = abs(x_start - x_end)
            delta_y = self.caves.cave_depth - len(self.caves.caves_list[cave_num]) + 1
            amphipod_num = self.caves.caves_list[cave_num][-1]
        dist = delta_x + delta_y
        
        return dist * 10**amphipod_num

    def make_move(self, move):
        hall = self.hall.copy()
        caves_list = deepcopy(self.caves.caves_list)
        start = move[0].split('.')
        end = move[1].split('.')
        if start[0] == 'h':
            hall_pos = int(start[1])
            cave_num = int(end[1])
            hall[hall_pos] = 0
            caves_list[cave_num].append(cave_num)
        if start[0] == 'c':
            hall_pos = int(end[1])
            cave_num = int(start[1])
            amphipod_num = caves_list[cave_num].pop()
            hall[hall_pos] = amphipod_num + 1
            
        burrow_state = (hall, caves_list)
        return burrow_state
    
    def hall_empty(self):
        return self.hall.sum() == 0
    
    def everyone_home(self):
        return self.caves.ready.count(False) == 0
    
    def __str__(self):
        hall_str = ''.join([str(x) for x in self.hall])
        all_caves = []
        for cave in self.caves.caves_list:
            all_caves.extend(cave)
        cave_str = ''.join([str(x) for x in all_caves])
        
        return hall_str + cave_str

In [4]:
class Game:
    
    def __init__(self, burrow_state, energy_used, cave_depth):
        self.burrow = Burrow(burrow_state, cave_depth)
        self.energy_used = energy_used
        self.h = self._h()
        self.hash_string = None
        self.cave_depth = cave_depth
    
    def get_moves(self):
        moves = []
        moves = moves + self.burrow.go_home_moves()
        moves = moves + self.burrow.go_hall_moves()
        
        return moves
                       
    def win(self):
        return self.burrow.hall_empty() and self.burrow.everyone_home()
    
    def make_move(self, move):
        move_energy = self.burrow.move_cost(move)
        burrow_state = self.burrow.make_move(move)
        return Game(burrow_state, self.energy_used + move_energy, cave_depth=self.cave_depth)
    
    def __lt__(self, other):
        return (self.energy_used + self.h) < (other.energy_used + other.h)
    
    def _h(self):
        # TODO: Implement heuristic for A* (if needed)
        return 0
    
    def __str__(self):
        if self.hash_string is None:
            self.hash_string = str(self.burrow) + str(self.energy_used)
        return self.hash_string

In [5]:
# Star 1
# caves = [['B', 'C'],  # right-most is at cave exit
#         ['A', 'A'],
#         ['B', 'D'],
#         ['C', 'D']]

# Star 2
caves = [['B', 'D', 'D', 'C'],  # right-most is at cave exit
        ['A', 'B', 'C', 'A'],
        ['B', 'A', 'B', 'D'],
        ['C', 'C', 'A', 'D']]

caves_list = [[ord(x) - ord('A') for x in cave] for cave in caves]
hall = np.zeros(11, dtype=np.int8)
burrow_state = (hall, caves_list)
cave_depth = len(burrow_state[1][0])

In [6]:
open_set = []
closed_set = set()
game = Game(burrow_state, 0, cave_depth)

heapq.heappush(open_set, game)
done = False
while len(open_set) > 0 and not done:
    g = heapq.heappop(open_set)
    if str(g) in closed_set:
        continue
    if g.win():
        break
    for move in g.get_moves():
        gnew = g.make_move(move)
        heapq.heappush(open_set, gnew)
    closed_set.add(str(g))
    
print(f'Minimum energy = {g.energy_used}')

Minimum energy = 41284
